In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [9]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_xyzConcurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [16]:
payload = {"prompt": "Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu).", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [17]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [18]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [19]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [14]:
@my_decorator
def inference(thread, payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
    print(f"Prompt {thread}==> {payload_data['prompt']} \n {response.text}")
    return response, payload_data['prompt'], thread
    
        

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    print(len(prompts))
    global df
    for i in range(1):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
            futures = [executor.submit(inference, srno, {**payload, 'prompt': prompt}) for srno,prompt in enumerate(prompts, start=0)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-ef3fe643023b43b5a62aaa491db449d1","object":"text_completion","created":520753,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 4.561717 seconds 


Prompt 6==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-c6e9714a8f3e401288f4c8f177d01717","object":"text_completion","created":520753,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapsho

Prompt 7==> Problem: Write a function in python to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-a92cc5a369154405828b67cd8802c82b","object":"text_completion","created":520754,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef binary_search(arr, target):\n    left, right = 0, len(arr) - 1\n    while left <= right:\n        mid = (left + right) // 2\n        if arr[mid] == target:\n            return mid\n        elif arr[mid] < target:\n            left = mid + 1\n        else:\n            right = mid - 1\n    return -1\n\n# Test the function\narr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\ntarget = 5\nprint(binary_search(arr, target))  # Output: 4\n\ntarget = 11\nprint(binary_search(arr, target))  # Output: -1","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":47

Execution time: 16.503159 seconds 


Execution time: 16.767785 seconds 

Execution time: 16.797973 seconds 


Execution time: 16.456897 seconds 



Execution time: 16.479211 seconds 


Prompt 41==> Problem: Write a function in Ruby to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-7d73f3c07aff4c3182fc892a7de857aa","object":"text_completion","created":520768,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s)\n  s.reverse\nend\n\nputs reverse_string(\"Hello World\")","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":32,"total_tokens":62,"completion_tokens":30}}
Execution time: 5.180184 seconds 


Prompt 40==> Problem: Write a function in Ruby to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-230863897f144debaabe1cf0c678

Execution time: 11.270289 seconds 


Prompt 35==> Problem: Write a function in javascript to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-e7ff6689bdce46118d3034c9e989fdd6","object":"text_completion","created":520762,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence is a series of numbers in which each number is the sum of the two preceding ones, usually starting with 0 and 1. That is,\n\nFibonacci(0) = 0,\nFibonacci(1) = 1,\nFibonacci(n) = Fibonacci(n - 1) + Fibonacci(n - 2) for n > 1.\n\nHere is a simple implementation of the function in JavaScript:\n\n```javascript\nfunction fibonacci(n) {\n    if (n <= 1) {\n        return n;\n    } else {\n        return fibonacci(n - 1) + fibonacci(n - 2);\n    }\n}\n```\n\nThis function uses recursion to calculate the nth Fibonacci number. However, this","logpr

Prompt 62==> Problem: Build a function in CSS to toggle between light and dark modes for a website.Input: None.Output: None (update the website theme). 
 {"id":"cmpl-342a04f0e367458ab231a6943eaf05f2","object":"text_completion","created":520771,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple CSS function to toggle between light and dark modes:\n\n```css\n@media (prefers-color-scheme: dark) {\n  body {\n    background-color: #333;\n    color: #fff;\n  }\n}\n\n@media (prefers-color-scheme: light) {\n  body {\n    background-color: #fff;\n    color: #333;\n  }\n}\n```\n\nThis CSS function uses the `prefers-color-scheme` media feature to detect the user's preferred color scheme. If the user's preferred color scheme is dark, the body background color will be set to a dark grey (#333) and the text color will be set to white (#fff). If the user's preferred color scheme is light, t

Execution time: 16.772730 seconds 



Prompt 63==> Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu). 
 {"id":"cmpl-01d91c99f6b644edafe1818f2e96cb66","object":"text_completion","created":520773,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can create an animated dropdown menu using CSS and JavaScript.\n\nHTML:\n```html\n<button id=\"dropdownBtn\">Dropdown</button>\n<div id=\"dropdownMenu\">\n  <a href=\"#\">Link 1</a>\n  <a href=\"#\">Link 2</a>\n  <a href=\"#\">Link 3</a>\n</div>\n```\n\nCSS:\n```css\n#dropdownMenu {\n  display: none;\n  position: absolute;\n  background-color: #f9f9f9;\n  min-width: 160px;\n  box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2);\n  z-index: 1;\n}\n","logprobs":null,"finish_reason":"leng

Prompt 70==> Problem: Implement an autocomplete feature in react for a search input field.Input: String, user input in the search field.Output: None (update the autocomplete suggestions). 
 {"id":"cmpl-399692e9f1074c31a1b3b6564debc32b","object":"text_completion","created":520780,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can implement an autocomplete feature in React using a controlled component and the useState hook.\n\n```jsx\nimport React, { useState } from 'react';\n\nconst Autocomplete = () => {\n  const [inputValue, setInputValue] = useState('');\n  const [suggestions, setSuggestions] = useState([]);\n\n  const handleInputChange = (event) => {\n    const userInput = event.target.value;\n\n    // Filter suggestions based on user input\n    const filteredSuggestions = suggestions.filter(\n      (suggestion) =>\n        suggestion.toLowerCase().i

In [15]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    index=0
    global df
    for i in range(1):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
            futures = [executor.submit(inference, srno, payload) for srno in range(100)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-f3c65729fed346f39fc4674a9e705cac","object":"text_completion","created":520352,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 8.118660 seconds 


Prompt 4==> Problem: Write a function in python to check if two 

Prompt 25==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-342c920d2e6f4784ba643e441a2eb74a","object":"text_completion","created":520353,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Prompt 29==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicat

Execution time: 7.728093 seconds 


Execution time: 7.605895 seconds 


Execution time: 8.052507 seconds 


Execution time: 7.545251 seconds 

Execution time: 7.825602 seconds 



Execution time: 8.250600 seconds 


Execution time: 7.640745 seconds 


Execution time: 8.151742 seconds 


Execution time: 7.583045 seconds 

Execution time: 7.704802 seconds 



Prompt 30==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-4f203ab7fd1a4bb0ac2a8424fc7c3486","object":"text_completion","created":520360,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the func


Execution time: 7.941365 seconds 

Execution time: 8.571286 seconds 

Execution time: 8.291329 seconds 



Execution time: 8.533391 seconds 


Execution time: 8.407189 seconds 




Execution time: 8.274467 seconds 


Execution time: 8.718619 seconds 


Execution time: 8.113958 seconds 


Execution time: 8.330332 seconds 


Execution time: 8.357188 seconds 

Execution time: 8.330998 seconds 



Prompt 60==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-35bde0b5907648419f139e4988d9a300","object":"text_completion","created":520367,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorte

Execution time: 8.450169 seconds 


Execution time: 8.299783 seconds 


Execution time: 8.828971 seconds 


Prompt 90==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-63c8d8e23f164b1299539102fab91171","object":"text_completion","created":520375,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 6.317636 se